# TFM
### Ismael Franco Hernando

# Imports

In [1]:
import sys, os, cv2, glob, pickle
import scipy.ndimage
import plotly.express as px
import numpy as np
import pandas as pd
from skimage import io
import seaborn as sns
import scipy as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image
from statistics import mean
import matplotlib.image
import matplotlib.pyplot
from tqdm import tqdm
from mysql.connector import (connection)
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Conexión y pruebas en la base de datos

In [2]:
try:
    cnx = connection.MySQLConnection(user='leer', 
                                     password='candanal',
                                     host='138.100.82.178',
                                     database='deepquality')

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrecto")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)

In [3]:
q01 = "SELECT count(*) FROM GR_LPValues"
r01 = pd.read_sql(q01,cnx)
r01

,count(*)
0,115972


In [4]:
q02 = "SELECT count(*) FROM GR_QPValues"
r02 = pd.read_sql(q02,cnx)
r02

,count(*)
0,1272312


In [5]:
q03 = "SELECT count(*) FROM Gauges"
r03 = pd.read_sql(q03,cnx)
r03

,count(*)
0,6


In [6]:
q04 = "SELECT count(*) FROM V_Coils"
r04 = pd.read_sql(q04,cnx)
r04

,count(*)
0,1160


# Bobinas clasificadas

In [7]:
q05 = "SELECT * FROM V_Coils"
clasif = pd.read_sql(q05,cnx)
clasif

,id,SID,PLANT,NAME,STARTTIME,MATERIAL,LENGTH,WIDTH,THICK,WEIGHT,BASEPID,CLASSLABEL,Label,ZnMin,ZnMax
0,1161,226031456,1066.0,226031456,2022-01-28,0.0,792.0,1183.0,3.0,22.0,220100.0,NOK,0,60,80
1,1162,226101621,1066.0,226101621,2022-01-31,0.0,273.0,1252.0,4.0,10.0,220100.0,NOK,2,215,258
2,1163,226120037,1066.0,226120037,2022-01-31,0.0,599.0,1305.0,2.0,15.0,220100.0,OK,1,155,175
3,1164,225387481,1066.0,225387481,2022-01-13,0.0,268.0,1502.0,5.0,15.0,220100.0,OK,1,60,80
4,1165,225406471,1066.0,225406471,2022-01-13,0.0,735.0,1158.0,2.0,19.0,220100.0,NOK,0,45,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,2316,233642421,1066.0,233642421,2022-08-31,0.0,576.0,1422.0,3.0,24.0,220800.0,OK,2,138,158
1156,2317,233445180,1066.0,233445180,2022-08-25,0.0,513.0,1437.0,4.0,23.0,220800.0,OK,1,125,145
1157,2318,232624927,1066.0,232624927,2022-08-01,0.0,345.0,1482.0,2.0,11.0,220800.0,OK,2,155,175
1158,2319,232635130,1066.0,232635130,2022-08-01,0.0,457.0,968.0,3.0,11.0,220800.0,OK,1,88,108


# Imágenes 1D


In [8]:
q06 = "SELECT * FROM GR_LPValues"
unoD = pd.read_sql(q06,cnx)
unoD

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
0,228318255,116480.0,124.0,1066.0,T,143.32179,143.32179,143.32179,1.0
1,228318255,116480.0,201.0,1066.0,T,145.10339,145.10339,145.10339,1.0
2,228318255,116480.0,202.0,1066.0,T,138.98370,138.98370,138.98370,1.0
3,228318255,116480.0,123.0,1066.0,T,145.78580,145.78580,145.78580,1.0
4,228318255,119040.0,124.0,1066.0,T,148.77290,148.77290,148.77290,1.0
...,...,...,...,...,...,...,...,...,...
115967,232800547,0.0,123.0,1066.0,T,207.00020,207.00020,207.00020,1.0
115968,232800547,5120.0,124.0,1066.0,T,89.30820,89.30820,89.30820,1.0
115969,232800547,5120.0,201.0,1066.0,T,248.35600,248.35600,248.35600,1.0
115970,232800547,5120.0,202.0,1066.0,T,76.74000,76.74000,76.74000,1.0


In [75]:
unoD = unoD.sort_values(['COILID']).reset_index(drop=True)

In [78]:
def analizador(datos, clasificados):
    rDF = pd.DataFrame(columns = ['COILID' , 'TILEID', 'MID', 'MEAN' , 'ZnMax', 'ZnMin', 
                                  'datosMax', 'datosMin', 'CLASSLABEL', 'LabelVerificada', 'Fallos'])
    sid = -1
    znMax = -1
    znMin = -1
    label = 'null'
    fallosTotales = 0
    fallosAntes = 0
    for i in range(len(datos)):
        id = datos['COILID'][i]
        if sid != id:
            fallosTotales = 0
            fallosAntes = 0
            registro = clasificados.loc[clasificados['SID']==id]
            sid = registro['SID'].iloc[0]
            znMax = registro['ZnMax'].iloc[0]
            znMin = registro['ZnMin'].iloc[0]
            label = registro['CLASSLABEL'].iloc[0]
        
        datosMax = datos['MAX'][i]
        datosMin = datos['MIN'][i]
        tileid = datos['TILEID'][i]
        mid = datos['MID'][i]
        mean = datos['MEAN'][i]
        
        verificado = 'OK'
        if (round(datosMax,1) > znMax or round(datosMin,1) < znMin) and (datosMax > 0 and datosMin > 0):
            verificado = 'NOK'
            fallosTotales = fallosTotales + 1
            
        rDF.loc[i] = [id, tileid, mid, mean, znMax, znMin, datosMax, datosMin, label, verificado, fallosTotales]
        
        if fallosTotales > fallosAntes:
            fallosAntes = fallosTotales
            rDF.loc[rDF.COILID==id, 'Fallos'] = fallosTotales
            
        
    
    return rDF.sort_values(['COILID','MID', 'TILEID'])
            
            

In [79]:
aDF = analizador(unoD, clasif)
aDF

,COILID,TILEID,MID,MEAN,ZnMax,ZnMin,datosMax,datosMin,CLASSLABEL,LabelVerificada,Fallos
4,225216688,0.0,123.0,88.94490,158,138,88.94490,88.94490,OK,NOK,56
8,225216688,5120.0,123.0,81.65457,158,138,81.65457,81.65457,OK,NOK,56
23,225216688,10240.0,123.0,94.79793,158,138,94.79793,94.79793,OK,NOK,56
15,225216688,15360.0,123.0,127.82180,158,138,127.82180,127.82180,OK,NOK,56
75,225216688,20480.0,123.0,125.88000,158,138,125.88000,125.88000,OK,NOK,56
...,...,...,...,...,...,...,...,...,...,...,...
115935,233667056,109056.0,202.0,51.92412,70,50,51.92412,51.92412,OK,OK,51
115939,233667056,114176.0,202.0,52.02400,70,50,52.02400,52.02400,OK,OK,51
115902,233667056,119552.0,202.0,48.61295,70,50,48.61295,48.61295,OK,NOK,51
115899,233667056,124672.0,202.0,42.18800,70,50,42.18800,42.18800,OK,NOK,51


In [80]:
#Serialización del DF
aDF.to_pickle("./serializados/unoD.pkl") 

In [83]:
aDF.head(60)

,COILID,TILEID,MID,MEAN,ZnMax,ZnMin,datosMax,datosMin,CLASSLABEL,LabelVerificada,Fallos
4,225216688,0.0,123.0,88.94490,158,138,88.94490,88.94490,OK,NOK,56
8,225216688,5120.0,123.0,81.65457,158,138,81.65457,81.65457,OK,NOK,56
23,225216688,10240.0,123.0,94.79793,158,138,94.79793,94.79793,OK,NOK,56
15,225216688,15360.0,123.0,127.82180,158,138,127.82180,127.82180,OK,NOK,56
75,225216688,20480.0,123.0,125.88000,158,138,125.88000,125.88000,OK,NOK,56
96,225216688,25600.0,123.0,131.03960,158,138,131.03960,131.03960,OK,NOK,56
51,225216688,30720.0,123.0,141.06149,158,138,141.06149,141.06149,OK,OK,56
58,225216688,35840.0,123.0,128.91940,158,138,128.91940,128.91940,OK,NOK,56
54,225216688,40960.0,123.0,129.99760,158,138,129.99760,129.99760,OK,NOK,56
19,225216688,46080.0,123.0,136.57680,158,138,136.57680,136.57680,OK,NOK,56


# Imágenes 2D

In [14]:
q07 = "SELECT * FROM GR_QPValues"
dosD = pd.read_sql(q07,cnx)
dosD

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
0,227501982,123817.0,1243.0,1066.0,T,126.82590,126.82590,126.82590,1.0
1,227501982,123817.0,1234.0,1066.0,T,184.95621,184.95621,184.95621,1.0
2,227501982,125353.0,1243.0,1066.0,T,125.99920,125.99920,125.99920,1.0
3,227501982,125353.0,1234.0,1066.0,T,183.72990,183.72990,183.72990,1.0
4,227501982,126889.0,1243.0,1066.0,T,134.66490,134.66490,134.66490,1.0
...,...,...,...,...,...,...,...,...,...
1272307,231628391,56.0,1243.0,1066.0,T,299.99670,299.99670,299.99670,1.0
1272308,231628391,1592.0,1243.0,1066.0,T,299.99670,299.99670,299.99670,1.0
1272309,231628391,3384.0,1243.0,1066.0,T,299.08820,299.08820,299.08820,1.0
1272310,231628391,5176.0,1243.0,1066.0,T,290.91199,290.91199,290.91199,1.0


In [15]:
dosD.loc[dosD['COILID']=='226101621']

,COILID,TILEID,MID,PLANT,SIDE,MEAN,MAX,MIN,COUNT
879997,226101621,4721.0,1234.0,1066.0,T,269.89349,269.89349,269.89349,1.0
879998,226101621,4721.0,1243.0,1066.0,T,110.86780,110.86780,110.86780,1.0
879999,226101621,113.0,1234.0,1066.0,T,196.01401,196.01401,196.01401,1.0
880000,226101621,113.0,1243.0,1066.0,T,204.98100,204.98100,204.98100,1.0
880001,226101621,14421.0,1234.0,1066.0,T,282.21359,282.21359,282.21359,1.0
...,...,...,...,...,...,...,...,...,...
1041101,226101621,120035.0,1234.0,1066.0,T,163.22861,163.22861,163.22861,1.0
1041102,226101621,124899.0,1243.0,1066.0,T,192.71240,192.71240,192.71240,1.0
1041103,226101621,124899.0,1234.0,1066.0,T,163.18350,163.18350,163.18350,1.0
1041104,226101621,129763.0,1243.0,1066.0,T,192.71240,192.71240,192.71240,1.0


In [63]:
aDF2 = analizador(dosD, clasif)
aDF2

,COILID,TILEID,MID,MEAN,ZnMax,ZnMin,datosMax,datosMin,CLASSLABEL,LabelVerificada,Fallos
1,227501982,123817.0,1234.0,184.95621,175,155,184.95621,184.95621,OK,NOK,5
3,227501982,125353.0,1234.0,183.72990,175,155,183.72990,183.72990,OK,NOK,5
0,227501982,123817.0,1243.0,126.82590,175,155,126.82590,126.82590,OK,NOK,5
2,227501982,125353.0,1243.0,125.99920,175,155,125.99920,125.99920,OK,NOK,5
4,227501982,126889.0,1243.0,134.66490,175,155,134.66490,134.66490,OK,NOK,5
18,227522012,115541.0,1234.0,76.45282,80,60,76.45282,76.45282,OK,OK,26
36,227522012,115598.0,1234.0,71.38795,80,60,71.38795,71.38795,OK,OK,26
20,227522012,117333.0,1234.0,78.13152,80,60,78.13152,78.13152,OK,OK,26
34,227522012,117390.0,1234.0,71.59948,80,60,71.59948,71.59948,OK,OK,26
6,227522012,119352.0,1234.0,80.72434,80,60,80.72434,80.72434,OK,NOK,26
